<img src='./img/LogoWekeo_Copernicus_RGB_0.png' alt='Logo EU Copernicus EUMETSAT' align='right' width='20%'></img>

<br>

# Functions

This notebook lists all `functions` that are defined and used throughout the `LTPy course`.
The following functions are listed:

**[Data loading and re-shaping functions](#load_reshape)**
* [generate_geographical_subset](#generate_geographical_subset)
* [select_channels_for_rgb](#rgb_channels)
* [normalize](#normalize)

**[Data visualization functions](#visualization)**
* [visualize_pcolormesh](#visualize_pcolormesh)
* [visualize_s3_pcolormesh](#visualize_s3_pcolormesh)


<hr>

#### Load required libraries

In [1]:
import os
from matplotlib import pyplot as plt

import xarray as xr
from netCDF4 import Dataset
import numpy as np
import glob

from matplotlib import pyplot as plt
import matplotlib.colors
from matplotlib.colors import LogNorm
import cartopy.crs as ccrs
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
import cartopy.feature as cfeature

import warnings
warnings.simplefilter(action = "ignore", category = RuntimeWarning)
warnings.simplefilter(action = "ignore", category = FutureWarning)

<hr>

## <a id="load_reshape"></a>Data loading and re-shaping functions

### <a id='generate_geographical_subset'></a>`generate_geographical_subset`

In [1]:
def generate_geographical_subset(xarray, latmin, latmax, lonmin, lonmax):
    """ 
    Generates a geographical subset of a xarray DataArray and shifts the longitude grid from a 0-360 to a -180 to 180 deg grid.
    
    Parameters:
        xarray (xarray DataArray): a xarray DataArray with latitude and longitude coordinates
        latmin, latmax, lonmin, lonmax (int): boundaries of the geographical subset
        
    Returns:
        Geographical subset of a xarray DataArray.
    """   
    return xarray.where((xarray.latitude < latmax) & (xarray.latitude > latmin) & (xarray.longitude < lonmax) & (xarray.longitude > lonmin),drop=True)

### <a id='rgb_channels'></a> `select_channels_for_rgb`

In [ ]:
def select_channels_for_rgb(xarray, red_channel, green_channel, blue_channel):
    """ 
    Selects the channels / bands of a multi-dimensional xarray for red, green and blue composites.
    
    Parameters:
        xarray(xarray Dataset): xarray Dataset object that stores the different channels / bands.
        red_channel(str): Name of red channel to be selected
        green_channel(str): Name of green channel to be selected
        blue_channel(str): Name of blue channel to be selected

    Returns:
        Three xarray DataArray objects with selected channels / bands
    """  
    return xarray[red_channel], xarray[green_channel], xarray[blue_channel]

### <a id='normalize'></a> `normalize`

In [ ]:
def normalize(array):
    """ 
    Normalizes a numpy array / xarray DataArray object value to values between 0 and 1.
    
    Parameters:
        xarray(numpy array or xarray DataArray): xarray DataArray or numpy array object.

    Returns:
        Normalized array
    """ 
    array_min, array_max = array.min(), array.max()
    return ((array - array_min)/(array_max - array_min))

<hr>

## <a id="visualization"></a>Data visualization functions

### <a id='visualize_pcolormesh'></a>`visualize_pcolormesh`

In [3]:
def visualize_pcolormesh(data_array, longitude, latitude, projection, color_scale, unit, long_name, vmin, vmax, lonmin, lonmax, latmin, latmax, log=True, set_global=True):
    """ 
    Visualizes a numpy array with matplotlib's 'pcolormesh' function.
    
    Parameters:
        data_array: any numpy MaskedArray, e.g. loaded with the NetCDF library and the Dataset function
        longitude: numpy Array holding longitude information
        latitude: numpy Array holding latitude information
        projection: a projection provided by the cartopy library, e.g. ccrs.PlateCarree()
        color_scale (str): string taken from matplotlib's color ramp reference
        unit (str): the unit of the parameter, taken from the NetCDF file if possible
        long_name (str): long name of the parameter, taken from the NetCDF file if possible
        vmin (int): minimum number on visualisation legend
        vmax (int): maximum number on visualisation legend
        lonmin,lonmax,latmin,latmax: geographic extent of the plot
        log (logical): set True, if the values shall be represented in a logarithmic scale
        set_global (logical): set True, if the plot shall have a global coverage
    """
    fig=plt.figure(figsize=(20, 10))

    ax = plt.axes(projection=projection)

    # define the coordinate system that the grid lons and grid lats are on
    if(log):
        img = plt.pcolormesh(longitude, latitude, np.squeeze(data_array), norm=LogNorm(), 
                             cmap=plt.get_cmap(color_scale), transform=ccrs.PlateCarree(),
                            vmin=vmin,
                            vmax=vmax,
                            shading='gouraud')
    else:
        img = plt.pcolormesh(longitude, latitude, data_array, 
                        cmap=plt.get_cmap(color_scale), transform=ccrs.PlateCarree(),
                        vmin=vmin,
                        vmax=vmax,
                        shading='gouraud')

    ax.add_feature(cfeature.BORDERS, edgecolor='black', linewidth=1)
    ax.add_feature(cfeature.COASTLINE, edgecolor='black', linewidth=1)

    if (projection==ccrs.PlateCarree()):
        ax.set_extent([lonmin, lonmax, latmin, latmax], projection)
        gl = ax.gridlines(draw_labels=True, linestyle='--')
        gl.xformatter=LONGITUDE_FORMATTER
        gl.yformatter=LATITUDE_FORMATTER
        gl.xlabel_style={'size':14}
        gl.ylabel_style={'size':14}

    if(set_global):
        ax.set_global()
        ax.gridlines()

    cbar = fig.colorbar(img, ax=ax, orientation='horizontal', fraction=0.04, pad=0.1)
    cbar.set_label(unit, fontsize=16)
    cbar.ax.tick_params(labelsize=14)
    ax.set_title(long_name, fontsize=20, pad=40.0)

 #   plt.show()
    return fig, ax

### <a id='visualize_s3_pcolormesh'></a>`visualize_s3_pcolormesh`

In [1]:
def visualize_s3_pcolormesh(color_array, array, latitude, longitude, title):
    """ 
    Visualizes a numpy array (Sentinel-3 data) with matplotlib's 'pcolormesh' function as RGB image.
    
    Parameters:
        color_array (numpy MaskedArray): any numpy MaskedArray, e.g. loaded with the NetCDF library and the Dataset function
        array (xarray.DataArray): in order to get the array dimensions
        longitude (numpy Array): array with longitude values
        latitude (numpy Array) : array with latitude values
        title (str): title of the resulting plot
    """
    fig=plt.figure(figsize=(20, 12))

    ax=plt.axes(projection=ccrs.Mercator())
    ax.coastlines()

    gl = ax.gridlines(draw_labels=True, linestyle='--')
    gl.xformatter=LONGITUDE_FORMATTER
    gl.yformatter=LATITUDE_FORMATTER
    gl.xlabel_style={'size':14}
    gl.ylabel_style={'size':14}

    img1 = plt.pcolormesh(longitude, latitude, array*np.nan, color=color_array,
                          clip_on = True,
                          edgecolors=None,
                          zorder=0,
                          transform=ccrs.PlateCarree())
    ax.set_title(title, fontsize=20, pad=40.0)
    plt.show()

<hr>

<img src='./img/all_partners_wekeo.png' alt='Logo EU Copernicus EUMETSAT' align='right' width='100%'></img>